#### Objective
To clean, de-duplicate, classify, and structurally expand Hazardous Waste Annual Return data for accurate analysis and reporting, 
while preserving fiscal-year logic and authorization-wise details.

#### Brief Description

This script is applied to the Hazardous Waste Generating Industries – Annual Return file and performs two sequential processing tasks, 
producing two structured Excel outputs.

#### Final Outputs

This script generates two Excel files:
> Task 1 Output

* Cleaned, de-duplicated, FY-segregated datasets
* Suitable for validation, compliance checks, and reporting

> Task 2 Output

* Authorization-wise expanded datasets
* Suitable for aggregation, analysis, and database ingestion

In [21]:
#!/usr/bin/env python3
"""
Jupyter-ready full script — comparison columns chosen from original header list,
so helper columns won't break duplicate / variant detection.

Paste entire cell into Jupyter and run. Edit input_path, out1, out2 at bottom.
"""

import os
import re
import traceback
from datetime import datetime
import pandas as pd
import numpy as np
from openpyxl.utils import column_index_from_string

pd.options.mode.chained_assignment = None

def find_unique_id_col(cols):
    for c in cols:
        if isinstance(c, str) and ('unique' in c.lower() and 'id' in c.lower()):
            return c
    return None

def colname_by_excel_letter(df, letter):
    idx = column_index_from_string(letter) - 1
    if idx < 0 or idx >= len(df.columns):
        raise IndexError(f"Excel column {letter} -> index {idx} out of range (len cols={len(df.columns)}).")
    return df.columns[idx]

def normalize_for_compare(df):
    out = df.copy()
    for c in out.select_dtypes(include=['object']).columns:
        out[c] = out[c].astype(str).str.strip()
        out.loc[out[c].isin(['nan', 'None', 'NA', '']), c] = pd.NA
    return out

def parse_submitted_on_value(val):
    raw = val
    if pd.isna(val):
        return (pd.NaT, None, val)
    s = str(val).strip()
    if s == '':
        return (pd.NaT, None, val)
    s = re.sub(r'[\x00-\x1f]+', ' ', s).strip()
    # remove millisecond chunk before AM/PM like ":637PM"
    s = re.sub(r':\d{1,6}(?=\s*[AP]M$)', '', s, flags=re.IGNORECASE)
    # ensure space before AM/PM if missing
    s = re.sub(r'(?i)(\d{1,2}:\d{2}:\d{2})(AM|PM)$', r'\1 \2', s)
    dt = pd.to_datetime(s, errors='coerce')
    if pd.isna(dt):
        fmts = [
            "%b %d %Y %I:%M:%S %p",
            "%d-%b-%Y %H:%M:%S",
            "%Y-%m-%d %H:%M:%S",
            "%d/%m/%Y %H:%M:%S",
        ]
        for f in fmts:
            try:
                dt = pd.to_datetime(datetime.strptime(s, f))
                break
            except Exception:
                dt = pd.NaT
    display = dt.strftime("%Y-%m-%d %H:%M:%S") if not pd.isna(dt) else s
    return (dt, display, raw)

def explode_columns_positionwise(df, cols_to_explode):
    out_rows = []
    if df.shape[0] == 0:
        return pd.DataFrame(columns=df.columns)
    for _, row in df.iterrows():
        splits = {}
        max_len = 0
        for col in cols_to_explode:
            val = row.get(col, "")
            if pd.isna(val) or str(val).strip() == "":
                parts = []
            else:
                parts = [x.strip() for x in str(val).split(',') if x.strip() != ""]
            splits[col] = parts
            max_len = max(max_len, len(parts))
        if max_len == 0:
            out_rows.append(row.to_dict())
            continue
        for i in range(max_len):
            new_row = row.copy()
            for col in cols_to_explode:
                vals = splits[col]
                new_row[col] = vals[i] if i < len(vals) else pd.NA
            out_rows.append(new_row.to_dict())
    return pd.DataFrame(out_rows, columns=df.columns)

def make_safe_sheet_name(name, existing):
    MAX = 31
    base = name[:MAX]
    candidate = base
    i = 1
    while candidate in existing:
        suffix = f"_{i}"
        candidate = (base[:MAX - len(suffix)]) + suffix
        i += 1
        if i > 999:
            raise RuntimeError("Too many duplicate sheet names")
    existing.add(candidate)
    return candidate

def main(input_path, out_task1_path, out_task2_path):
    print(f"Loading '{input_path}' ...")
    df = pd.read_excel(input_path, sheet_name=0, header=0, engine='openpyxl')
    original_cols = list(df.columns)  # capture original headers immediately

    unique_col = find_unique_id_col(original_cols)
    if unique_col:
        print(f"Detected Unique ID column: '{unique_col}' (will be ignored during duplicate checks).")
    else:
        print("No Unique id column detected automatically. All columns (except Submitted on) will be used for comparisons.")

    try:
        submitted_on_col = colname_by_excel_letter(df, 'B')
        submission_year_col = colname_by_excel_letter(df, 'AH')
    except Exception as e:
        raise RuntimeError(f"Failed to map Excel letters to columns: {e}")

    print(f"Using Submitted-on column: '{submitted_on_col}' (Excel B)")
    print(f"Using Submission-Year column: '{submission_year_col}' (Excel AH)")

    # parse Submitted-on robustly
    raw_submitted = df[submitted_on_col].copy()
    parsed_list = []
    display_list = []
    parsed_count = 0
    for v in raw_submitted:
        dt, disp, raw = parse_submitted_on_value(v)
        parsed_list.append(dt)
        display_list.append(disp)
        if not pd.isna(dt):
            parsed_count += 1
    print(f"Parsed {parsed_count}/{len(raw_submitted)} Submitted-on values.")

    parsed_col = '_parsed_submitted_on'
    df[parsed_col] = pd.to_datetime(pd.Series(parsed_list))
    df['_display_submitted_on'] = display_list

    # mark original index for stable selection of "first" duplicate
    df['_orig_index'] = range(len(df))

    # --- Build comparison column lists FROM original_cols so helper cols aren't included ---
    # exclude Unique id and Submitted on
    exclude_for_comp = [submitted_on_col, '_orig_index']
    if unique_col:
        exclude_for_comp.append(unique_col)

    cols_for_comp = [c for c in original_cols if c not in exclude_for_comp]
    # cols_for_comp now contains only original columns used for comparisons (submission_year_col still included)
    print(f"Number of columns used for comparisons (original headers minus Unique id & Submitted on): {len(cols_for_comp)}")

    # normalized df for comparisons
    df_for_comp_norm = normalize_for_compare(df[cols_for_comp].copy())

    # ----- Submission Year Variants (identical except Submission Year) -----
    # Build key excluding submission_year_col (and submitted_on is already excluded)
    cols_except_year = [c for c in cols_for_comp if c != submission_year_col]
    # create tuple key for every row
    df['_key_except_year'] = df_for_comp_norm[cols_except_year].astype(object).apply(lambda row: tuple(row.values), axis=1)
    # group
    groups_except_year = df.groupby('_key_except_year')

    variant_indices = []
    for k, g in groups_except_year:
        unique_years = g[submission_year_col].astype(str).dropna().unique()
        if len(unique_years) > 1:
            variant_indices.extend(g.index.tolist())

    variant_df = df.loc[variant_indices].copy().drop(columns=['_key_except_year'])
    print(f"Submission Year Variants rows found: {len(variant_df)}")

    # Remove variants before duplicate detection
    df_remaining = df.drop(index=variant_indices).copy().drop(columns=['_key_except_year'])

    # ----- Complete duplicates (identical across all comparison columns) -----
    # comp_cols: use original_cols minus Unique id and Submitted on (same as cols_for_comp)
    comp_cols = cols_for_comp.copy()
    # Normalize df_remaining for comp_cols (note: df_remaining may be smaller)
    df_norm_remaining = normalize_for_compare(df_remaining[comp_cols].astype(object))
    df_remaining['_key_all'] = df_norm_remaining.astype(object).apply(lambda row: tuple(row.values), axis=1)

    dup_groups = df_remaining.groupby('_key_all')
    complete_dup_indices = []
    keep_indices = []
    for k, g in dup_groups:
        if len(g) > 1:
            g_sorted = g.sort_values(by='_orig_index')
            keep_idx = g_sorted.index[0]
            other_idx = g_sorted.index[1:].tolist()
            keep_indices.append(keep_idx)
            complete_dup_indices.extend(other_idx)
        else:
            keep_indices.append(g.index[0])

    complete_duplicates_df = df_remaining.loc[complete_dup_indices].copy().drop(columns=['_key_all'])
    print(f"Complete duplicate rows found: {len(complete_duplicates_df)}")

    main_df = df_remaining.loc[keep_indices].copy().drop(columns=['_key_all'])

    # ----- Use parsed datetime for FY split -----
    fy_start = pd.Timestamp('2024-04-01')
    fy_end = pd.Timestamp('2025-03-31')

    main_df_fy = main_df[(main_df[parsed_col] >= fy_start) & (main_df[parsed_col] <= fy_end)].copy()
    main_df_outside = main_df[~((main_df[parsed_col] >= fy_start) & (main_df[parsed_col] <= fy_end))].copy()

    print(f"Rows in Main FY 24-25 (before display fix): {len(main_df_fy)}")
    print(f"Rows in Main Outside FY (before display fix): {len(main_df_outside)}")

    # Build readable Submitted-on for export: prefer parsed datetime formatted, else raw original
    def make_display_col_for_export(df_local):
        disp = []
        for idx, row in df_local.iterrows():
            p = row.get(parsed_col)
            if pd.notna(p):
                disp.append(pd.to_datetime(p).strftime("%Y-%m-%d %H:%M:%S"))
            else:
                orig = row.get(submitted_on_col)
                if pd.isna(orig):
                    disp.append(pd.NA)
                else:
                    disp.append(str(orig))
        return pd.Series(disp, index=df_local.index)

    for d in [main_df_fy, main_df_outside, complete_duplicates_df, variant_df]:
        d[submitted_on_col] = make_display_col_for_export(d)
        # drop helper cols if present
        for h in ['_parsed_submitted_on', '_display_submitted_on', '_orig_index']:
            if h in d.columns:
                d.drop(columns=[h], inplace=True, errors='ignore')

    # Prepare safe sheet names
    existing = set()
    sheet_names_task1 = {
        'main_fy': make_safe_sheet_name('Main FY 24-25', existing),
        'main_out': make_safe_sheet_name('Main Outside FY', existing),
        'dup': make_safe_sheet_name('Complete Duplicates', existing),
        'variants': make_safe_sheet_name('YearVariants', existing)
    }

    print(f"Writing Task 1 workbook to '{out_task1_path}' ...")
    with pd.ExcelWriter(out_task1_path, engine='xlsxwriter', datetime_format='yyyy-mm-dd') as writer:
        main_df_fy.to_excel(writer, sheet_name=sheet_names_task1['main_fy'], index=False)
        main_df_outside.to_excel(writer, sheet_name=sheet_names_task1['main_out'], index=False)
        complete_duplicates_df.to_excel(writer, sheet_name=sheet_names_task1['dup'], index=False)
        variant_df.to_excel(writer, sheet_name=sheet_names_task1['variants'], index=False)
    print("Task 1 complete.")

    # ---------------- TASK 2: position-wise split BD..BH ----------------
    letters = ['BD', 'BE', 'BF', 'BG', 'BH']
    cols_to_explode = []
    for L in letters:
        try:
            cname = colname_by_excel_letter(df, L)
            cols_to_explode.append(cname)
        except Exception:
            print(f"Warning: Column {L} not present (skipping).")

    print("Columns to position-wise split:", cols_to_explode)

    existing2 = set()
    sheet_names_task2 = {
        'fy_exp': make_safe_sheet_name('FY 24-25 Expanded', existing2),
        'out_exp': make_safe_sheet_name('Outside FY Expanded', existing2),
        'dup_exp': make_safe_sheet_name('Duplicates Expanded', existing2),
        'var_exp': make_safe_sheet_name('YearVars Expanded', existing2)
    }

    print(f"Creating Task 2 workbook: '{out_task2_path}' ...")
    with pd.ExcelWriter(out_task2_path, engine='xlsxwriter', datetime_format='yyyy-mm-dd') as writer:
        expanded_fy = explode_columns_positionwise(main_df_fy.copy(), cols_to_explode)
        expanded_fy.to_excel(writer, sheet_name=sheet_names_task2['fy_exp'], index=False)

        expanded_out = explode_columns_positionwise(main_df_outside.copy(), cols_to_explode)
        expanded_out.to_excel(writer, sheet_name=sheet_names_task2['out_exp'], index=False)

        expanded_cd = explode_columns_positionwise(complete_duplicates_df.copy(), cols_to_explode)
        expanded_cd.to_excel(writer, sheet_name=sheet_names_task2['dup_exp'], index=False)

        expanded_sv = explode_columns_positionwise(variant_df.copy(), cols_to_explode)
        expanded_sv.to_excel(writer, sheet_name=sheet_names_task2['var_exp'], index=False)

    print("Task 2 complete. Files written:")
    print(" -", out_task1_path)
    print(" -", out_task2_path)

    # Extra debug: sample some groups that were flagged as duplicates / variants (first few)
    if len(complete_duplicates_df) > 0:
        print("\nSample complete-duplicate Unique IDs (moved to Complete Duplicates):")
        sn = complete_duplicates_df.head(5)
        if unique_col in sn.columns:
            print(sn[unique_col].tolist())
        else:
            print("No Unique id column; showing first rows of duplicates:")
            print(sn.head().to_dict(orient='records'))
    if len(variant_df) > 0:
        print("\nSample Year-Variant Unique IDs (moved to Year Variants):")
        sn = variant_df.head(5)
        if unique_col in sn.columns:
            print(sn[unique_col].tolist())
        else:
            print(sn.head().to_dict(orient='records'))


# -------------------- DIRECT RUN FOR JUPYTER --------------------
input_path = r'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/Annual_Return_24-25.xlsx'
out1 = r'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task1_output_C_fix.xlsx'
out2 = r'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task2_output_C_fix.xlsx'

try:
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input file not found: {input_path}")
    print("Starting processing...")
    main(input_path, out1, out2)
    print("Done.")
except Exception:
    print("ERROR during processing:")
    traceback.print_exc()


Starting processing...
Loading 'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/Annual_Return_24-25.xlsx' ...
Detected Unique ID column: 'Unique id' (will be ignored during duplicate checks).
Using Submitted-on column: 'Submited on' (Excel B)
Using Submission-Year column: 'Submission year' (Excel AH)
Parsed 18310/18310 Submitted-on values.
Number of columns used for comparisons (original headers minus Unique id & Submitted on): 62
Submission Year Variants rows found: 106
Complete duplicate rows found: 5
Rows in Main FY 24-25 (before display fix): 8954
Rows in Main Outside FY (before display fix): 9245
Writing Task 1 workbook to 'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task1_output_C_fix.xlsx' ...
Task 1 complete.
Columns to position-wise split: ['Type of waste dispatched', 'Quantity of waste dispatched', 'Unit dispatched', 'Dispatched to', 'Dispatched to facility']
Creating Task 2 workbook: 'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task2_output_C_fix.xlsx' ...
Task 2 complete. Files

In [25]:
#!/usr/bin/env python3
"""
Jupyter-ready: position-wise expansion with district (AJ) kept on first expanded row
and blanked on subsequent expanded rows.

Paste the entire cell into Jupyter and run. Edit input_path, out1, out2 at bottom.
"""

import os
import re
import traceback
from datetime import datetime
import pandas as pd
import numpy as np
from openpyxl.utils import column_index_from_string

pd.options.mode.chained_assignment = None

def find_unique_id_col(cols):
    for c in cols:
        if isinstance(c, str) and ('unique' in c.lower() and 'id' in c.lower()):
            return c
    return None

def colname_by_excel_letter(df, letter):
    idx = column_index_from_string(letter) - 1
    if idx < 0 or idx >= len(df.columns):
        raise IndexError(f"Excel column {letter} -> index {idx} out of range (len cols={len(df.columns)}).")
    return df.columns[idx]

def normalize_for_compare(df):
    out = df.copy()
    for c in out.select_dtypes(include=['object']).columns:
        out[c] = out[c].astype(str).str.strip()
        out.loc[out[c].isin(['nan', 'None', 'NA', '']), c] = pd.NA
    return out

def parse_submitted_on_value(val):
    raw = val
    if pd.isna(val):
        return (pd.NaT, None, val)
    s = str(val).strip()
    if s == '':
        return (pd.NaT, None, val)
    s = re.sub(r'[\x00-\x1f]+', ' ', s).strip()
    s = re.sub(r':\d{1,6}(?=\s*[AP]M$)', '', s, flags=re.IGNORECASE)
    s = re.sub(r'(?i)(\d{1,2}:\d{2}:\d{2})(AM|PM)$', r'\1 \2', s)
    dt = pd.to_datetime(s, errors='coerce')
    if pd.isna(dt):
        fmts = [
            "%b %d %Y %I:%M:%S %p",
            "%d-%b-%Y %H:%M:%S",
            "%Y-%m-%d %H:%M:%S",
            "%d/%m/%Y %H:%M:%S",
        ]
        for f in fmts:
            try:
                dt = pd.to_datetime(datetime.strptime(s, f))
                break
            except Exception:
                dt = pd.NaT
    display = dt.strftime("%Y-%m-%d %H:%M:%S") if not pd.isna(dt) else s
    return (dt, display, raw)

def explode_columns_positionwise(df, cols_to_explode, blank_cols=None):
    """
    Position-wise explosion with optional blanking of specific columns in expanded rows,
    but preserves blank_cols value on the first generated row (i==0). Subsequent generated
    rows (i >= 1) will have blank_cols set to NaN.

    Parameters:
      df: DataFrame to expand
      cols_to_explode: list of column names to split (mapped from BD..BH)
      blank_cols: list of column names to blank in subsequent generated rows (not the first)
    """
    out_rows = []
    if df.shape[0] == 0:
        return pd.DataFrame(columns=df.columns)
    blank_cols = blank_cols or []

    for _, row in df.iterrows():
        splits = {}
        max_len = 0
        for col in cols_to_explode:
            val = row.get(col, "")
            if pd.isna(val) or str(val).strip() == "":
                parts = []
            else:
                parts = [x.strip() for x in str(val).split(',') if x.strip() != ""]
            splits[col] = parts
            max_len = max(max_len, len(parts))

        # no splits -> keep original row unchanged (including district)
        if max_len == 0:
            out_rows.append(row.to_dict())
            continue

        # create expanded rows; keep blank_cols on first row, blank them on subsequent rows
        for i in range(max_len):
            new_row = row.copy()
            for col in cols_to_explode:
                vals = splits[col]
                new_row[col] = vals[i] if i < len(vals) else pd.NA
            # for i==0 keep original value; for i>=1 set blank_cols to NaN
            if i >= 1:
                for bc in blank_cols:
                    if bc in new_row.index:
                        new_row[bc] = pd.NA
            out_rows.append(new_row.to_dict())

    return pd.DataFrame(out_rows, columns=df.columns)

def make_safe_sheet_name(name, existing):
    MAX = 31
    base = name[:MAX]
    candidate = base
    i = 1
    while candidate in existing:
        suffix = f"_{i}"
        candidate = (base[:MAX - len(suffix)]) + suffix
        i += 1
        if i > 999:
            raise RuntimeError("Too many duplicate sheet names")
    existing.add(candidate)
    return candidate

def main(input_path, out_task1_path, out_task2_path):
    print(f"Loading '{input_path}' ...")
    df = pd.read_excel(input_path, sheet_name=0, header=0, engine='openpyxl')
    original_cols = list(df.columns)  # capture original headers immediately

    unique_col = find_unique_id_col(original_cols)
    if unique_col:
        print(f"Detected Unique ID column: '{unique_col}' (will be ignored during duplicate checks).")
    else:
        print("No Unique id column detected automatically. All columns (except Submitted on) will be used for comparisons.")

    try:
        submitted_on_col = colname_by_excel_letter(df, 'B')
        submission_year_col = colname_by_excel_letter(df, 'AH')
    except Exception as e:
        raise RuntimeError(f"Failed to map Excel letters to columns: {e}")

    print(f"Using Submitted-on column: '{submitted_on_col}' (Excel B)")
    print(f"Using Submission-Year column: '{submission_year_col}' (Excel AH)")

    # parse Submitted-on robustly
    raw_submitted = df[submitted_on_col].copy()
    parsed_list = []
    display_list = []
    parsed_count = 0
    for v in raw_submitted:
        dt, disp, raw = parse_submitted_on_value(v)
        parsed_list.append(dt)
        display_list.append(disp)
        if not pd.isna(dt):
            parsed_count += 1
    print(f"Parsed {parsed_count}/{len(raw_submitted)} Submitted-on values.")

    parsed_col = '_parsed_submitted_on'
    df[parsed_col] = pd.to_datetime(pd.Series(parsed_list))
    df['_display_submitted_on'] = display_list

    # mark original index for stable selection of "first" duplicate
    df['_orig_index'] = range(len(df))

    # --- Build comparison column lists FROM original_cols so helper cols aren't included ---
    exclude_for_comp = [submitted_on_col, '_orig_index']
    if unique_col:
        exclude_for_comp.append(unique_col)

    cols_for_comp = [c for c in original_cols if c not in exclude_for_comp]
    print(f"Number of columns used for comparisons (original headers minus Unique id & Submitted on): {len(cols_for_comp)}")

    df_for_comp_norm = normalize_for_compare(df[cols_for_comp].copy())

    # Identify Submission Year Variants
    cols_except_year = [c for c in cols_for_comp if c != submission_year_col]
    df['_key_except_year'] = df_for_comp_norm[cols_except_year].astype(object).apply(lambda row: tuple(row.values), axis=1)
    groups_except_year = df.groupby('_key_except_year')

    variant_indices = []
    for k, g in groups_except_year:
        unique_years = g[submission_year_col].astype(str).dropna().unique()
        if len(unique_years) > 1:
            variant_indices.extend(g.index.tolist())

    variant_df = df.loc[variant_indices].copy().drop(columns=['_key_except_year'])
    print(f"Submission Year Variants rows found: {len(variant_df)}")

    # Remove variants before duplicate detection
    df_remaining = df.drop(index=variant_indices).copy().drop(columns=['_key_except_year'])

    # Complete duplicates detection
    comp_cols = cols_for_comp.copy()
    df_norm_remaining = normalize_for_compare(df_remaining[comp_cols].astype(object))
    df_remaining['_key_all'] = df_norm_remaining.astype(object).apply(lambda row: tuple(row.values), axis=1)

    dup_groups = df_remaining.groupby('_key_all')
    complete_dup_indices = []
    keep_indices = []
    for k, g in dup_groups:
        if len(g) > 1:
            g_sorted = g.sort_values(by='_orig_index')
            keep_idx = g_sorted.index[0]
            other_idx = g_sorted.index[1:].tolist()
            keep_indices.append(keep_idx)
            complete_dup_indices.extend(other_idx)
        else:
            keep_indices.append(g.index[0])

    complete_duplicates_df = df_remaining.loc[complete_dup_indices].copy().drop(columns=['_key_all'])
    print(f"Complete duplicate rows found: {len(complete_duplicates_df)}")

    main_df = df_remaining.loc[keep_indices].copy().drop(columns=['_key_all'])

    # FY split using parsed submitted-on
    fy_start = pd.Timestamp('2024-04-01')
    fy_end = pd.Timestamp('2025-03-31')

    main_df_fy = main_df[(main_df[parsed_col] >= fy_start) & (main_df[parsed_col] <= fy_end)].copy()
    main_df_outside = main_df[~((main_df[parsed_col] >= fy_start) & (main_df[parsed_col] <= fy_end))].copy()

    print(f"Rows in Main FY 24-25 (before display fix): {len(main_df_fy)}")
    print(f"Rows in Main Outside FY (before display fix): {len(main_df_outside)}")

    # Prepare readable Submitted-on for export
    def make_display_col_for_export(df_local):
        disp = []
        for idx, row in df_local.iterrows():
            p = row.get(parsed_col)
            if pd.notna(p):
                disp.append(pd.to_datetime(p).strftime("%Y-%m-%d %H:%M:%S"))
            else:
                orig = row.get(submitted_on_col)
                if pd.isna(orig):
                    disp.append(pd.NA)
                else:
                    disp.append(str(orig))
        return pd.Series(disp, index=df_local.index)

    for d in [main_df_fy, main_df_outside, complete_duplicates_df, variant_df]:
        d[submitted_on_col] = make_display_col_for_export(d)
        for h in ['_parsed_submitted_on', '_display_submitted_on', '_orig_index']:
            if h in d.columns:
                d.drop(columns=[h], inplace=True, errors='ignore')

    # Write Task 1 with safe sheet names
    existing = set()
    sheet_names_task1 = {
        'main_fy': make_safe_sheet_name('Main FY 24-25', existing),
        'main_out': make_safe_sheet_name('Main Outside FY', existing),
        'dup': make_safe_sheet_name('Complete Duplicates', existing),
        'variants': make_safe_sheet_name('YearVariants', existing)
    }

    print(f"Writing Task 1 workbook to '{out_task1_path}' ...")
    with pd.ExcelWriter(out_task1_path, engine='xlsxwriter', datetime_format='yyyy-mm-dd') as writer:
        main_df_fy.to_excel(writer, sheet_name=sheet_names_task1['main_fy'], index=False)
        main_df_outside.to_excel(writer, sheet_name=sheet_names_task1['main_out'], index=False)
        complete_duplicates_df.to_excel(writer, sheet_name=sheet_names_task1['dup'], index=False)
        variant_df.to_excel(writer, sheet_name=sheet_names_task1['variants'], index=False)
    print("Task 1 complete.")

    # Map BD..BH columns and district AJ
    letters = ['BD', 'BE', 'BF', 'BG', 'BH']
    cols_to_explode = []
    for L in letters:
        try:
            cname = colname_by_excel_letter(df, L)
            cols_to_explode.append(cname)
        except Exception:
            print(f"Warning: Column {L} not present (skipping).")

    # district column name from AJ (if present)
    try:
        district_col_name = colname_by_excel_letter(df, 'AJ')
        print(f"Detected district column (AJ): '{district_col_name}'")
    except Exception:
        district_col_name = None
        print("District column AJ not found; expanded rows will not be blanked for district.")

    print("Columns to position-wise split:", cols_to_explode)

    # Prepare safe sheet names for Task 2
    existing2 = set()
    sheet_names_task2 = {
        'fy_exp': make_safe_sheet_name('FY 24-25 Expanded', existing2),
        'out_exp': make_safe_sheet_name('Outside FY Expanded', existing2),
        'dup_exp': make_safe_sheet_name('Duplicates Expanded', existing2),
        'var_exp': make_safe_sheet_name('YearVars Expanded', existing2)
    }

    print(f"Creating Task 2 workbook: '{out_task2_path}' ...")
    with pd.ExcelWriter(out_task2_path, engine='xlsxwriter', datetime_format='yyyy-mm-dd') as writer:
        # pass district_col_name as blank_cols so expanded rows will have district blanked except first generated row
        expanded_fy = explode_columns_positionwise(main_df_fy.copy(), cols_to_explode, blank_cols=[district_col_name] if district_col_name else None)
        expanded_fy.to_excel(writer, sheet_name=sheet_names_task2['fy_exp'], index=False)

        expanded_out = explode_columns_positionwise(main_df_outside.copy(), cols_to_explode, blank_cols=[district_col_name] if district_col_name else None)
        expanded_out.to_excel(writer, sheet_name=sheet_names_task2['out_exp'], index=False)

        expanded_cd = explode_columns_positionwise(complete_duplicates_df.copy(), cols_to_explode, blank_cols=[district_col_name] if district_col_name else None)
        expanded_cd.to_excel(writer, sheet_name=sheet_names_task2['dup_exp'], index=False)

        expanded_sv = explode_columns_positionwise(variant_df.copy(), cols_to_explode, blank_cols=[district_col_name] if district_col_name else None)
        expanded_sv.to_excel(writer, sheet_name=sheet_names_task2['var_exp'], index=False)

    print("Task 2 complete. Files written:")
    print(" -", out_task1_path)
    print(" -", out_task2_path)

    # debug samples
    if len(complete_duplicates_df) > 0:
        print("\nSample complete-duplicate Unique IDs moved to Complete Duplicates:")
        sn = complete_duplicates_df.head(5)
        if unique_col in sn.columns:
            print(sn[unique_col].tolist())
        else:
            print(sn.head().to_dict(orient='records'))
    if len(variant_df) > 0:
        print("\nSample Year-Variant Unique IDs moved to YearVariants:")
        sn = variant_df.head(5)
        if unique_col in sn.columns:
            print(sn[unique_col].tolist())
        else:
            print(sn.head().to_dict(orient='records'))


# -------------------- DIRECT RUN FOR JUPYTER --------------------
input_path = r'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/Annual_Return_24-25.xlsx'
out1 = r'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task1_output_D_fix.xlsx'
out2 = r'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task2_output_D_fix.xlsx'

try:
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input file not found: {input_path}")
    print("Starting processing...")
    main(input_path, out1, out2)
    print("Done.")
except Exception:
    print("ERROR during processing:")
    traceback.print_exc()


Starting processing...
Loading 'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/Annual_Return_24-25.xlsx' ...
Detected Unique ID column: 'Unique id' (will be ignored during duplicate checks).
Using Submitted-on column: 'Submited on' (Excel B)
Using Submission-Year column: 'Submission year' (Excel AH)
Parsed 18310/18310 Submitted-on values.
Number of columns used for comparisons (original headers minus Unique id & Submitted on): 62
Submission Year Variants rows found: 106
Complete duplicate rows found: 5
Rows in Main FY 24-25 (before display fix): 8954
Rows in Main Outside FY (before display fix): 9245
Writing Task 1 workbook to 'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task1_output_D_fix.xlsx' ...
Task 1 complete.
Detected district column (AJ): 'District'
Columns to position-wise split: ['Type of waste dispatched', 'Quantity of waste dispatched', 'Unit dispatched', 'Dispatched to', 'Dispatched to facility']
Creating Task 2 workbook: 'D:/Techknowgreen/Rutuja_mam/12noV_Hazardous/task2_out